In [1]:
import pandas as pd
from google.cloud.bigquery import Client,  LoadJobConfig, WriteDisposition, SchemaField

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Product Info Dataframe

In [3]:
query = """
--This query will process 162.36 MB when run.
SELECT  
    product_id,
    ean_id,
    label,
    --activity,
    web_family,
    web_subfamily,
    web_subsubfamily,
    colors, 
    status,
    eur_regular_price,
    theme,
    --gamme,
    style,
    product_type,
    main_material,
    product_material,
    composition,
    --nb_places,
    --sofa_shape,
    --is_convertible,
    height,
    width,
    depth,
    weight,
FROM `mdm-data-prod.b_transform_products.last_products` 
WHERE is_marketplace_product = FALSE
AND web_subfamily = 'CHAISES'
"""

In [4]:
client = Client()

data_bq = client.query(query).to_dataframe()
data_bq.shape

/home/sarnal/Workspace_linux/hackaton-adk/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


(2701, 19)

In [5]:
data_chairs = pd.read_csv('../data/data_chaises.csv')
data_chairs.shape

(3534, 27)

In [6]:
df = data_bq.merge(data_chairs[['product_id', 'img_url', 'img_gcs_uri']], on='product_id', how='inner')
df.shape

(251, 21)

In [7]:
df.head(2)

,product_id,ean_id,label,web_family,web_subfamily,web_subsubfamily,colors,status,eur_regular_price,theme,style,product_type,main_material,product_material,composition,height,width,depth,weight,img_url,img_gcs_uri
0,234579,3611872345792,CHS BUR MAURICETTE TERRA PD G,CHAISES,CHAISES,CHAISES,Terracotta,Reconduit,97.94,None,Scandicraft - Contemporain,Assise de bureau,Acier,Velours,Produit fini simple,85.0,58.0,58.0,7.4,https://res.cloudinary.com/dammdm/image/privat...,gs://mdm-data-preprod-poc-product-similarity-e...
1,238841,3611872388416,CHS CLYDE GRIS ANTHRACITE_1,CHAISES,CHAISES,CHAISES,Anthracite | Bois moyen,Reconduit,45.02,None,Epuré - Contemporain,Assise à dîner,Acier,Polyester,Produit fini simple,86.0,44.0,55.0,4.5,https://res.cloudinary.com/dammdm/image/privat...,gs://mdm-data-preprod-poc-product-similarity-e...


In [8]:
df.web_family.value_counts(dropna=False)

web_family
CHAISES    251
Name: count, dtype: int64

In [9]:
df.status.value_counts(dropna=False)

status
Reconduit        213
Nouveauté         26
Non reconduit     10
None               2
Name: count, dtype: int64

In [10]:
df = df.loc[df.status.isin(['Reconduit', 'Nouveauté'])]
df.shape

(239, 21)

In [11]:
df.theme.value_counts(dropna=False)

theme
None    239
Name: count, dtype: int64

In [12]:
df.colors.value_counts(dropna=False)

colors
Beige                                                                                16
Ecru                                                                                 10
Noir                                                                                  9
Marron                                                                                8
Blanc                                                                                 8
Kaki                                                                                  7
Vert                                                                                  6
Beige | Noir                                                                          5
Bois clair                                                                            5
Beige | Bois moyen                                                                    5
Terracotta                                                                            4
Kaki | Noir              

In [13]:
df['style'].value_counts(dropna=False)

style
Chic - Classique              62
Scandicraft - Contemporain    50
Epuré - Contemporain          46
Tradi - Classique             32
Arty - Contemporain           22
Campagne - Classique          11
Neo indus - Autre              8
Bohème - Ethnique              4
Exo chic - Ethnique            3
Craft voyage - Ethnique        1
Name: count, dtype: int64

In [14]:
df.product_type.value_counts(dropna=False)

product_type
Assise à dîner      220
Assise de bureau     19
Name: count, dtype: int64

In [15]:
df['composition'].value_counts(dropna=False)

composition
Produit fini simple      232
Produit fini assemblé      7
Name: count, dtype: int64

In [16]:
df.product_material.value_counts(dropna=False)

product_material
Polyester                   91
Velours                     72
None                        24
Rotin                       17
Suédine / Textile enduit    17
Lin                         13
Cuir                         5
Name: count, dtype: int64

In [17]:
df.loc[df.product_material.isna()].main_material.value_counts(dropna=False)

main_material
Bois                  15
PP - Polypropylène     8
Acier                  1
Name: count, dtype: int64

In [18]:
df = df.rename(columns={'web_subsubfamily': 'category'})

In [19]:
df = df.drop(columns=[
    'web_family',  # not necessary for chairs only
    'web_subfamily',  # not necessary for chairs only
    'status',  # already pre-sorted
    'theme',  # empty
    'composition'  # not necessary for chairs only
    ])
df.shape

(239, 16)

In [20]:
df.isna().sum()

product_id            0
ean_id                0
label                 0
category              0
colors                0
eur_regular_price     0
style                 0
product_type          0
main_material         0
product_material     24
height                0
width                 0
depth                 0
weight                0
img_url               0
img_gcs_uri           0
dtype: int64

In [21]:
table_schema = [
    SchemaField("product_id", "STRING", mode="NULLABLE", description="Product identifier"),
    SchemaField("ean_id", "STRING", mode="NULLABLE", description="EAN of the product"),
    SchemaField("label", "STRING", mode="NULLABLE", description="Product name"),
    SchemaField("category", "STRING", mode="NULLABLE", description="Product category (ex: CHAISES)"),
    SchemaField("colors", "STRING", mode="NULLABLE", description="Product colors, separated by '|'"),
    SchemaField("eur_regular_price", "FLOAT", mode="NULLABLE", description="Current price of the product in eur without discount"),
    SchemaField("style", "STRING", mode="NULLABLE", description="Product styles separated by '|'"),
    SchemaField("product_type", "STRING", mode="NULLABLE", description="Type of product used by purchase department (ex: fauteuil à diner)"),
    SchemaField("main_material", "STRING", mode="NULLABLE", description="Product main material"),
    SchemaField("product_material", "STRING", mode="NULLABLE", description="Product material"),
    SchemaField("height", "FLOAT", mode="NULLABLE", description="Product height"),
    SchemaField("width", "FLOAT", mode="NULLABLE", description="Product width"),
    SchemaField("depth", "FLOAT", mode="NULLABLE", description="Product depth"),
    SchemaField("weight", "FLOAT", mode="NULLABLE", description="Product weight in grams"),
    SchemaField("img_url", "STRING", mode="NULLABLE", description="Image URL of the product"),
    SchemaField("img_gcs_uri", "STRING", mode="NULLABLE", description="Image GCS URI of the product"),
]


In [22]:
client = Client(project='data-sandbox-410808')
dataset_id = 'datascience_playground'
table_name = f'extract_chairs_adk'

table_ref = client.dataset(dataset_id).table(table_name)

job_config = LoadJobConfig()
job_config.write_disposition = WriteDisposition.WRITE_TRUNCATE
job_config.schema = table_schema

job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

/home/sarnal/Workspace_linux/hackaton-adk/.venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=data-sandbox-410808, location=EU, id=91fdf11e-e22f-4deb-8cd7-1b12f17993fb>

### Verbatims Dataframe

In [42]:
df_verbatims = pd.read_excel('../data/202505 semantique produit mai25.xlsx', sheet_name='résultats')
df_verbatims.shape

(6150, 44)

In [43]:
df_verbatims.columns

Index(['Product ID', 'Saison', 'Statut', 'Libellé', 'Activité', 'Famille',
       'Sous-famille', 'Sous-sous-famille', 'ID famille', 'ID sous-famille',
       'ID sous-sous-famille', 'Famille web', 'Sous-famille web',
       'Sous-sous-famille web', 'Gamme', 'Catégorie de prix',
       'Fournisseur principal', 'Nombre d'avis', 'VA HT', 'Quantités vendues',
       'Prix moyen', 'Note globale moyenne', 'Note qualité moyenne',
       'Engagement', 'Nb avis - Problème de livraison ou de service client',
       'Nb avis - Prix trop élevé',
       'Nb avis - Produit arrivé endommagé ou incomplet',
       'Nb avis - Problème de qualité', 'Nb avis - Problème de sécurité',
       'Nb avis - Autre', 'Nb avis - Pas de catégorie',
       'Pourcentage - problème de livraison ou de service client',
       'Pourcentage - prix trop élevé',
       'Pourcentage - produit arrivé endommagé ou incomplet',
       'Pourcentage - problème de qualité',
       'Pourcentage - problème de sécurité', 'Pourcentage 

In [44]:
df_verbatims = df_verbatims[['Product ID', 'Note globale moyenne', 'Note qualité moyenne', 'Avis traduits - au global', 'Synthèse globale']].rename(
    columns={
        'Product ID': 'product_id',
        'Note globale moyenne': 'global_rating',
        'Note qualité moyenne': 'quality_rating',
        'Avis traduits - au global': 'verbatims',
        'Synthèse globale': 'verbatim_synthesis'
    }
)

df_verbatims.product_id = df_verbatims.product_id.astype(str)
df_verbatims.shape

(6150, 5)

In [45]:
df = df[['product_id']].merge(
    df_verbatims, on='product_id', how='inner'
)
df.shape

(181, 5)

In [46]:
df.isna().sum()

product_id            0
global_rating         0
quality_rating        0
verbatims             0
verbatim_synthesis    0
dtype: int64

In [47]:
df.head(2)

,product_id,global_rating,quality_rating,verbatims,verbatim_synthesis
0,234579,3.50,3.75,"Very nice visual impact\nseat not deep enough,...",- Summary: The product has a nice visual impac...
1,238841,4.56,4.28,Very satisfied\ncomfortable and efficient\nGOO...,- Summary: Customers generally found the chair...


In [48]:
table_schema_reviews = [
    SchemaField("product_id", "STRING", mode="NULLABLE", description="Product identifier"),
    SchemaField("global_rating", "FLOAT", mode="NULLABLE", description="Average global rating of the product"),
    SchemaField("quality_rating", "FLOAT", mode="NULLABLE", description="Average quality rating of the product"),
    SchemaField("verbatims", "STRING", mode="NULLABLE", description="Product verbatims separated by '\n'"),
    SchemaField("verbatim_synthesis", "STRING", mode="NULLABLE", description="General summary of the product reviews, main positive and negative points"),
]


In [49]:
client = Client(project='data-sandbox-410808')
dataset_id = 'datascience_playground'
table_name = f'extract_chairs_reviews_adk'

table_ref = client.dataset(dataset_id).table(table_name)

job_config = LoadJobConfig()
job_config.write_disposition = WriteDisposition.WRITE_TRUNCATE
job_config.schema = table_schema_reviews


job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

/home/sarnal/Workspace_linux/hackaton-adk/.venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=data-sandbox-410808, location=EU, id=f881bca4-87f8-4994-91c3-86038ef8e595>